In [1]:
import requests
import csv
import os

API_TOKEN = '12abfbaacdab48bc8948ed6061925e1f'
BASE_URL = 'https://api.football-data.org/v2/'
HEADERS = {'X-Auth-Token': API_TOKEN}

def fetch_data(endpoint):
    """
    Fetch data from the specified API endpoint.
    Returns the JSON response if successful, or None if failed.
    """
    try:
        response = requests.get(BASE_URL + endpoint, headers=HEADERS)
        response.raise_for_status()  # Raises HTTPError for bad responses
        return response.json()
    except requests.exceptions.HTTPError as http_err:
        print(f"HTTP error occurred: {http_err}")
    except Exception as err:
        print(f"Other error occurred: {err}")
    return None

def save_to_csv(data, filename):
    """
    Save data to a CSV file.
    """
    if not data:
        print("No data provided to save.")
        return

    if not os.path.exists('output'):
        os.makedirs('output')  # Creating output directory

    filepath = os.path.join('output', filename)

    try:
        with open(filepath, 'w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(data[0].keys())  # Write headers
            for row in data:
                writer.writerow(row.values())  # Write rows
        print(f"Data saved to {filepath}")
    except Exception as e:
        print(f"Error saving data to CSV: {e}")

def main():
    endpoint = 'competitions/PL/teams'
    data = fetch_data(endpoint)
    if data and 'teams' in data:
        save_to_csv(data['teams'], 'epl_teams.csv')
    else:
        print("No team data found or failed to retrieve.")

# Run the main function
main()


Data saved to output/epl_teams.csv


In [2]:
import unittest
from unittest.mock import patch, mock_open
import requests
from io import StringIO

class TestAPIScript(unittest.TestCase):

    @patch('requests.get')
    def test_fetch_data_success(self, mock_get):
        # Mock a successful API response
        mock_get.return_value.status_code = 200
        mock_get.return_value.json.return_value = {"teams": [{"name": "Team1"}, {"name": "Team2"}]}

        endpoint = 'competitions/PL/teams'
        data = fetch_data(endpoint)

        self.assertIsNotNone(data)
        self.assertIn('teams', data)

    @patch('requests.get')
    def test_fetch_data_http_error(self, mock_get):
        # Mock a failed API response (404)
        mock_get.return_value.raise_for_status.side_effect = requests.exceptions.HTTPError("404 Error")

        endpoint = 'competitions/PL/teams'
        data = fetch_data(endpoint)

        self.assertIsNone(data)

    @patch('builtins.open', new_callable=mock_open)
    def test_save_to_csv(self, mock_file):
        # Test to save data to a CSV file
        data = [{"name": "Team1"}, {"name": "Team2"}]
        save_to_csv(data, 'test.csv')

        mock_file.assert_called_once_with('output/test.csv', 'w', newline='')
        mock_file().write.assert_any_call("name\n")  # Check that headers are written
        mock_file().write.assert_any_call("Team1\n")
        mock_file().write.assert_any_call("Team2\n")

    def test_save_to_csv_no_data(self):
        # Test with no data
        with patch('builtins.print') as mock_print:
            save_to_csv(None, 'test.csv')
            mock_print.assert_called_once_with("No data provided to save.")

# Run the tests
unittest.main(argv=[''], verbosity=2, exit=False)


test_fetch_data_http_error (__main__.TestAPIScript) ... ok
test_fetch_data_success (__main__.TestAPIScript) ... ok
test_save_to_csv (__main__.TestAPIScript) ... FAIL
test_save_to_csv_no_data (__main__.TestAPIScript) ... ok

FAIL: test_save_to_csv (__main__.TestAPIScript)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/usr/lib/python3.10/unittest/mock.py", line 1379, in patched
    return func(*newargs, **newkeywargs)
  File "<ipython-input-2-87dba9bd99a5>", line 37, in test_save_to_csv
    mock_file().write.assert_any_call("name\n")  # Check that headers are written
  File "/usr/lib/python3.10/unittest/mock.py", line 1000, in assert_any_call
    raise AssertionError(
AssertionError: write('name\n') call not found

----------------------------------------------------------------------
Ran 4 tests in 0.036s

FAILED (failures=1)


HTTP error occurred: 404 Error
Data saved to output/test.csv
